In [1]:
# -*- coding: utf-8 -*-
"""
各CSVファイルの tail_set 平均座標と、中心(640,360)からの距離を集計。
- DLC 3段ヘッダCSV (header=[0,1,2], index_col=0) を想定
- 低likelihood点を欠損として補間 → bfill/ffill → 0埋め（オプション）
- 出力: ファイルごとの平均 x/y、距離(平均/中央値/最大)、閾値超え判定(平均/任意時点)
"""

import os
import glob
import datetime
import numpy as np
import pandas as pd

# ====== 設定 ======
# 解析するCSVが入っているフォルダ（例：train用 or test用）
# 例1: INPUT_DIR = r"C:\...\data\train\train_csv"
# 例2: INPUT_DIR = r"C:\...\data\test\eval_csv"
INPUT_DIR = r"C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\train_csv"

# 中心座標と距離閾値
CENTER_X, CENTER_Y = 640.0, 360.0
DIST_THRESH = 367.0

# likelihood による低品質点の除外・補間を行うか
USE_LIKELIHOOD = True
MIN_KEEP_LIKELIHOOD = 0.6

# 解析対象のキーポイント名（DLC上の実名にマッチさせる。大文字小文字・空白/ハイフン/アンダースコアは無視して解決）
TAIL_NAME_REQUESTED = "tail set"

# 出力先: INPUT_DIR の親フォルダに保存（= train/ または test/ 直下に置く）
RUN_ID = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
OUT_CSV = os.path.join(os.path.dirname(INPUT_DIR), f"tailset_center_summary_{RUN_ID}.csv")


# ====== ユーティリティ ======
def _norm_name(s: str) -> str:
    """比較用に簡易正規化（小文字化＋空白/アンダースコア/ハイフン削除）"""
    return "".join(ch for ch in s.lower() if ch not in " _-")

def _resolve_bodypart_name(all_bodyparts, requested: str) -> str:
    """DLCに存在するボディパート実名を、requested（正規化）から解決"""
    norm2orig = {}
    for bp in all_bodyparts:
        k = _norm_name(bp)
        # 同名衝突は稀なので最初のものを採用
        if k not in norm2orig:
            norm2orig[k] = bp

    key = _norm_name(requested)
    if key not in norm2orig:
        raise ValueError(f"指定キーポイント '{requested}' が見つかりません。利用可能: {sorted(set(all_bodyparts))}")
    return norm2orig[key]

def _read_tail_xy(csv_path: str, tail_name_req: str, use_likelihood=True, min_keep_likelihood=0.6):
    """
    DLC 3段ヘッダCSVを読み込み、指定 tail_set の x/y ベクトル（長さT）を返す。
    低likelihoodは NaN → 線形補間 → bfill/ffill → 0埋め。
    """
    df = pd.read_csv(csv_path, header=[0,1,2], index_col=0)
    # 利用可能な bodyparts を列ヘッダから抽出
    bodyparts = list({bp for (_, bp, _) in df.columns})
    tail_name = _resolve_bodypart_name(bodyparts, tail_name_req)

    # x, y を取り出し
    try:
        x = df.xs((tail_name, "x"), level=[1,2], axis=1).values.flatten().astype(np.float32)
        y = df.xs((tail_name, "y"), level=[1,2], axis=1).values.flatten().astype(np.float32)
    except KeyError:
        raise KeyError(f"{os.path.basename(csv_path)} に '{tail_name}' の x/y 列が見つかりません。")

    if use_likelihood:
        try:
            lk = df.xs((tail_name, "likelihood"), level=[1,2], axis=1).values.flatten().astype(np.float32)
            low = lk < float(min_keep_likelihood)
            x = x.astype(np.float32); y = y.astype(np.float32)
            x[low] = np.nan
            y[low] = np.nan
        except KeyError:
            # likelihood 列が無い場合はそのまま進む
            pass

        # 線形補間 → bfill/ffill → 0埋め
        x = pd.Series(x).interpolate(method="linear", limit_direction="both").bfill().ffill().fillna(0.0).values
        y = pd.Series(y).interpolate(method="linear", limit_direction="both").bfill().ffill().fillna(0.0).values

    return x, y  # shape: (T,), (T,)


# ====== メイン処理 ======
def main():
    csv_paths = sorted(glob.glob(os.path.join(INPUT_DIR, "*.csv")))
    if not csv_paths:
        raise FileNotFoundError(f"CSVが見つかりません: {INPUT_DIR}\\*.csv")

    rows = []
    for p in csv_paths:
        try:
            x, y = _read_tail_xy(
                p,
                tail_name_req=TAIL_NAME_REQUESTED,
                use_likelihood=USE_LIKELIHOOD,
                min_keep_likelihood=MIN_KEEP_LIKELIHOOD
            )
        except Exception as e:
            print(f"[WARN] 読み込みスキップ: {os.path.basename(p)} -> {e}")
            continue

        n = len(x)
        mean_x = float(np.nanmean(x))
        mean_y = float(np.nanmean(y))

        # フレームごとの中心からの距離
        d_all = np.sqrt((x - CENTER_X) ** 2 + (y - CENTER_Y) ** 2)
        mean_d = float(np.nanmean(d_all))
        med_d  = float(np.nanmedian(d_all))
        max_d  = float(np.nanmax(d_all))

        rows.append({
            "file": os.path.basename(p),
            "n_frames": n,
            "tail_mean_x": round(mean_x, 3),
            "tail_mean_y": round(mean_y, 3),
            "center_x": CENTER_X,
            "center_y": CENTER_Y,
            "dist_mean": round(mean_d, 3),
            "dist_median": round(med_d, 3),
            "dist_max": round(max_d, 3),
            # 判別列（どれを使うか運用で選べます）
            "mean_gt_367": mean_d > DIST_THRESH,      # 平均距離が閾値超
            "any_frame_gt_367": bool((d_all > DIST_THRESH).any()),  # どこかの時点で閾値超
        })

    if not rows:
        raise RuntimeError("有効なCSVからデータを取得できませんでした。tail_set 名称の不一致などを疑ってください。")

    df = pd.DataFrame(rows).sort_values(["mean_gt_367", "dist_mean", "dist_max"], ascending=[False, False, False])
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

    # 画面にも概況
    flagged_mean = df[df["mean_gt_367"]]
    flagged_any  = df[df["any_frame_gt_367"]]
    print("\n=== Saved ===")
    print(OUT_CSV)
    print("\n=== mean_gt_367 = True (平均距離で閾値超) ===")
    print(flagged_mean[["file", "dist_mean", "dist_max"]].to_string(index=False) if not flagged_mean.empty else "(none)")
    print("\n=== any_frame_gt_367 = True (どこかの時点で閾値超) ===")
    print(flagged_any[["file", "dist_mean", "dist_max"]].to_string(index=False) if not flagged_any.empty else "(none)")

if __name__ == "__main__":
    main()



=== Saved ===
C:\kanno\vscode\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\data\train\tailset_center_summary_20251216-144734.csv

=== mean_gt_367 = True (平均距離で閾値超) ===
                                                 file  dist_mean  dist_max
    ivdd2_46DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_14DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_16DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_21DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_22DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_30DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
   normal_39DLC_resnet50_IvddOct30shuffle1_100000.csv    734.302   734.302
 ivdd1_case3DLC_resnet50_IvddOct30shuffle1_100000.csv    606.098   856.548
ivdd2_case11DLC_resnet50_IvddOct30shuffle1_100000.csv    573.229  1207.581
  